In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split

from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
# metadata 
# print(adult.metadata) 
  
# variable information 
# print(adult.variables) 

In [68]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [69]:
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "salary",
]

In [70]:
def clean_string(s):
    """
    Helper function that strips leading / trailing whitespace, lower
    cases, and replaces hyphens with underscores.
    """
    return s.strip().lower().replace("-", "_")


def parse_native_country(country):
    """
    Group countries other than United-States and Mexico into single
    "other" category"
    """
    country = clean_string(country)
    if country == "united_states" or country == "mexico":
        return country
    return "other"

In [71]:
train = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        header=None,
        na_values=[" ?"],
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        salary=lambda df: (df.salary == " >50K").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [72]:
test = (
    pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
        header=None,
        na_values=[" ?"],
        skiprows=1,
        names=names,
    )
    .drop(columns=["fnlwgt", "education_num"])
    # drop all rows with missing values
    .dropna()
    .reset_index(drop=True)
    # simple preprocessing on columns
    .assign(
        # clean all string columns
        education=lambda df: df.education.map(clean_string),
        marital_status=lambda df: df.marital_status.map(clean_string),
        occupation=lambda df: df.occupation.map(clean_string),
        race=lambda df: df.race.map(clean_string),
        relationship=lambda df: df.relationship.map(clean_string),
        workclass=lambda df: df.workclass.map(clean_string),
        # clean and aggregate native_country
        native_country=lambda df: df.native_country.map(parse_native_country),
        # encode binary features as integers
        # note extra '.' in test set not present in train set
        salary=lambda df: (df.salary == " >50K.").astype(np.int32),
        sex=lambda df: (df.sex == " Male").astype(np.int32),
    )
)

In [73]:
assert set(train.education) == set(test.education)
assert set(train.race) == set(test.race)
assert set(train.relationship) == set(test.relationship)
assert set(train.marital_status) == set(test.marital_status)

In [74]:
one_hot_features = [
    "workclass",
    "education",
    "occupation",
    "race",
    "relationship",
    "marital_status",
    "native_country",
]

cts_features = ["age", "capital_gain", "capital_loss", "hours_per_week"]

binary_features = ["sex", "salary"]

In [75]:
train_df = pd.concat(
    [train, pd.get_dummies(train.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

test_df = pd.concat(
    [test, pd.get_dummies(test.loc[:, one_hot_features], dtype=np.int32)],
    axis=1,
)

In [76]:
assert train_df.columns.tolist() == test_df.columns.tolist()

In [77]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [82]:
original_features = cts_features + one_hot_features + binary_features

train_df[original_features].to_csv('data/train.csv')
val_df[original_features].to_csv('data/val.csv')
test_df[original_features].to_csv('data/test.csv')

In [83]:
ss = StandardScaler()

train_df[cts_features] = ss.fit_transform(train_df[cts_features])
val_df[cts_features] = ss.transform(val_df[cts_features])
test_df[cts_features] = ss.transform(test_df[cts_features])

In [84]:
train_df.drop(columns=one_hot_features).to_csv('data/train-one-hot.csv')
val_df.drop(columns=one_hot_features).to_csv('data/val-one-hot.csv')
test_df.drop(columns=one_hot_features).to_csv('data/test-one-hot.csv')
